<a href="https://www.kaggle.com/code/dbtmddn41/prompt-recovery-hf-tpu-train2?scriptVersionId=173092327" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
class CFG:
    seed = 42
    dataset_path = "/kaggle/input/llm-prompt-recovery"
    preset = "/kaggle/input/gemma/transformers/1.1-7b-it/1"  # name of pretrained Gemma
#     preset = "/kaggle/input/llama-2/pytorch/7b-chat-hf/1"
    sequence_length = 1024 # max size of input sequence for training
    train_batch = 8 # size of the input batch in training
    validation_batch = 16
    epochs = 3 # number of epochs to train
    test_size = 0.
    train_datas = ['kishanvavdara', 'newtonbaba12345_3', 'newtonbaba12345_1', 'aatiffraz', 'newtonbaba12345_2', 'host', 'huosiyuan_2']
    validation_datas = ["winddude", 'juanmerinobermejo']
    lora_rank=4

In [2]:
!pip3 install transformers datasets peft -Uq
# !pip install torch~=2.1.0 --index-url https://download.pytorch.org/whl/cpu -q # Updating torch since we need the latest version
# !pip install torch_xla[tpu]~=2.1.0 -f https://storage.googleapis.com/libtpu-releases/index.html -q
!pip uninstall tensorflow -y # If we don't do this, TF will take over TPU and cause permission error for PT
!cp /kaggle/input/d/dbtmddn41/utils-xla/spmd_util.py . # From this repo: https://github.com/HeegyuKim/torch-xla-SPMD


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Successfully uninstalled tensorflow-2.15.0


In [3]:
import os
import pandas as pd
import numpy as np
import datasets
import torch.optim as optim
import torch_xla.debug.profiler as xp
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp # We also import mp modules if we wanna use that for some reason
import torch_xla.distributed.parallel_loader as pl
import torch_xla.test.test_utils as test_utils
import torch
import torch.nn as nn
import re
import torch_xla.experimental.xla_sharding as xs
import torch_xla.core.xla_model as xm
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, DataCollatorWithPadding, AutoConfig
) # You can use any of models with those configs (even flan T5 xxl!). Other models are not supported.

from transformers import logging as hf_logging
import torch.nn.functional as F
import torch_xla.runtime as xr

xr.use_spmd()

import torch_xla.experimental.xla_sharding as xs # "experimental" prefix always means you're gonna have a good time LMAO
from torch_xla.experimental.xla_sharded_tensor import XLAShardedTensor
from torch_xla.experimental.xla_sharding import Mesh

from peft import LoraConfig, TaskType, get_peft_model # If we wanna use peft. Quantazation requiers GPU though. You'll have to download already quantazed models
from spmd_util import partition_module                # You could experiment with using already quantazed models like 4bit/Llama-2-7b-Chat-GPTQ if you're feeling funny
from datasets import Dataset, load_dataset, concatenate_datasets
from dataclasses import dataclass
from tqdm import tqdm

from peft import LoftQConfig, LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
import datasets
import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.metrics import roc_auc_score

!export USE_TORCH=True #If we don't do this, transformers will seemingly bork the session upon import. Really weird error.
os.environ["PJRT_DEVICE"] = "TPU"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ.pop('TPU_PROCESS_ADDRESSES')
os.environ.pop('CLOUD_TPU_TASK_ID')
hf_logging.set_verbosity_error() # It can still display warnings which is a bit annoying but whatever


# MAX_INPUT=512
# MODEL = "/kaggle/input/gemma/transformers/7b-it/2" #You should be able to use 13B model with no changes! There should be enough HBM
device = xm.xla_device()

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_13/752609908.py:14: DeprecationWarning: Importing from `torch_xla.experimental.xla_sharding` will be deprecated after 2.2 release. Please use `torch_xla.distributed.spmd` instead.
  import torch_xla.experimental.xla_sharding as xs
/tmp/ipykernel_13/752609908.py:27: DeprecationWarning: Importing from `torch_xla.experimental.xla_sharded_tensor` will be deprecated after 2.2 release. Please use `torch_xla.distributed.spmd` instead.
  from torch_xla.experimental.xla_sharded_tensor import XLAShardedTensor


# Model

In [4]:

tokenizer = AutoTokenizer.from_pretrained(CFG.preset)
tokenizer.padding_side = 'right'
model = AutoModelForCausalLM.from_pretrained(CFG.preset, torch_dtype=torch.bfloat16)

if 'mistral' in CFG.preset:
    tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 4/4 [00:25<00:00,  6.35s/it]


In [5]:
lora_config = LoraConfig(
    r=CFG.lora_rank,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
    lora_dropout=0.05,
    bias="none"
#     inference_mode=True
)
peft_model = get_peft_model(model, lora_config)

In [6]:
config = AutoConfig.from_pretrained(CFG.preset)
num_devices = xr.global_runtime_device_count()
print(num_devices)
mesh_shape = (1, num_devices, 1, 1)
device_ids = np.array(range(num_devices))
mesh = Mesh(device_ids, mesh_shape, ('dp', 'fsdp', 'mp', 'sp'))
partition_module(peft_model, mesh) # After this, the model is sharded between cores but still has the same API as if it was on single device. Neat.


E0421 01:30:29.177498336      13 oauth2_credentials.cc:238]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {created_time:"2024-04-21T01:30:29.17748233+00:00", grpc_status:2}


8


In [7]:
# inputs = tokenizer("hello from the other side", return_tensors='pt')
# input_ids, attention_mask = inputs.input_ids.to(device), inputs.attention_mask.to(device)
# xs.mark_sharding(input_ids, mesh, (0, 1))
# xs.mark_sharding(attention_mask, mesh, (0, 1))
# output = peft_model(input_ids, attention_mask=attention_mask, labels=input_ids)
# output.loss

In [8]:
# import time
# for text in ["<s> [INST] Original Text:{}\nRewritten Text:{}\nWrite a prompt that was likely given to the LLM to rewrite original text into rewritten text. [/INST]".format(val_df.iloc[0]["original_text"], val_df.iloc[0]["rewritten_text"])]:
#     start_time = time.time()
#     inputs = tokenizer(text, return_tensors='pt')
#     input_ids, attention_mask = inputs.input_ids.to(device), inputs.attention_mask.to(device)
#     xs.mark_sharding(input_ids, mesh, (0, 1))
#     xs.mark_sharding(attention_mask, mesh, (0, 1))
#     output = peft_model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=500)
#     print(tokenizer.batch_decode(output))
#     print("걸린 시간: ", time.time() - start_time)

In [9]:
peft_model.print_trainable_parameters()

trainable params: 12,500,992 || all params: 9,336,613,888 || trainable%: 0.13389213852001589


# datasets

In [10]:
import pandas as pd
import numpy as np
import gc

In [11]:
%%time
df = pd.read_csv("/kaggle/input/all-in-one-dataset-with-embedding/df_with_emb_20240412.csv")

float_cols = df.select_dtypes('float64').columns
df[float_cols] = df[float_cols].astype('float32')
df = df[df['rewritten_text'].str.len() >= 20]
train_df = df[df['dataset_id'].isin(CFG.train_datas)].copy(deep=True)
val_df = df[df['dataset_id'].isin(CFG.validation_datas)].sample(n=1300).copy(deep=True)

# df = df.drop(index=0)
gc.collect()
train_df.head()

CPU times: user 1min 19s, sys: 4.64 s, total: 1min 24s
Wall time: 2min 15s


,original_text,rewrite_prompt,rewritten_text,dataset_id,original_text_emb_0,original_text_emb_1,original_text_emb_2,original_text_emb_3,original_text_emb_4,original_text_emb_5,...,rewritten_text_emb_758,rewritten_text_emb_759,rewritten_text_emb_760,rewritten_text_emb_761,rewritten_text_emb_762,rewritten_text_emb_763,rewritten_text_emb_764,rewritten_text_emb_765,rewritten_text_emb_766,rewritten_text_emb_767
0,"Dear Randy,\n\nI hope this letter finds you we...",Rephrase this letter to infuse it with an elfi...,"Dear Randy,\n\nMay this enchanted message find...",host,-0.028678,-0.072260,-0.003482,0.050595,-0.010600,-0.023786,...,-0.023307,0.011079,-0.026521,-0.035838,-0.011306,0.035690,-0.005810,0.004506,-0.037956,0.001139
14656,It'd been three days since Baldr Alwisson had ...,Rewrite this text in the style of a Victorian ...,\n\nHere is the text rewritten in the style of...,newtonbaba12345_1,-0.030413,-0.067652,0.049466,0.035858,0.008359,-0.015908,...,-0.033934,-0.027861,-0.033822,-0.052785,-0.000612,0.051786,0.012271,0.003726,-0.018933,-0.026539
14657,"I am the daemon that all men do seek, \n \n Du...",Translate the essence of this text into a ghos...,"\n\nSure, here's the ghost narrative:\n\nIn th...",newtonbaba12345_1,-0.014884,-0.078367,0.023980,0.069455,-0.034322,-0.023850,...,-0.001434,-0.021219,-0.001819,-0.044634,0.004461,0.038283,0.017500,0.011950,-0.003624,-0.032714
14658,"Come here children, listen to me explain \n Th...",Rewrite this text in the style of a ancient Ro...,"\n\nSure, here is the text rewritten in the st...",newtonbaba12345_1,-0.019390,-0.077755,0.044897,0.056840,-0.016640,-0.033011,...,-0.045966,0.003527,-0.030286,-0.054125,0.024585,0.031940,0.022491,-0.000578,-0.025836,-0.027405
14659,*I... Am? * \n \n You are. \n \n *I am. * \n \...,Translate the essence of this text into a the ...,\n\n**Roaring Twenties Narrative:**\n\nThe Roa...,newtonbaba12345_1,-0.009235,-0.055168,0.022361,0.036347,-0.000441,-0.013944,...,-0.033253,-0.031921,-0.006217,-0.032138,0.001856,0.040625,0.024044,0.026046,-0.044090,-0.009349


In [12]:
def truncate_txt(text, length):
    text_list = text.split()
    
    if len(text_list) <= length:
        return text
    
    return " ".join(text_list[:length])


def gen_val_prompt(df):
    
    # Truncate the texts to first 200 words for now
    # As we are having memory issues on Mixtral8x7b
    og_text = truncate_txt(df["original_text"].strip(), CFG.sequence_length//3)
    rewritten_text = truncate_txt(df["rewritten_text"].strip(), CFG.sequence_length//3)
    template = """<start_of_turn>user\nOriginal Text:{}\nRewritten Text:{}\nWrite a prompt that was likely given to the LLM to rewrite original text into rewritten text.<end_of_turn>
<start_of_turn>model
Output:
"""
    return template.format(og_text, rewritten_text).strip()

def gen_prompt(df):
    
    # Truncate the texts to first 200 words for now
    # As we are having memory issues on Mixtral8x7b
    og_text = truncate_txt(df["original_text"].strip(), CFG.sequence_length//3)
    rewritten_text = truncate_txt(df["rewritten_text"].strip(), CFG.sequence_length//3)
    rewrite_prompt = truncate_txt(df["rewrite_prompt"].strip(), CFG.sequence_length//3)
    template = """<start_of_turn>user\nOriginal Text:{}\nRewritten Text:{}\nWrite a prompt that was likely given to the LLM to rewrite original text into rewritten text.<end_of_turn>
<start_of_turn>model
Output: {}<end_of_turn><eos>"""
    return template.format(og_text, rewritten_text, rewrite_prompt).strip()

In [13]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from datasets import Dataset, DatasetDict

tqdm.pandas()
# ds = DatasetDict()
def preprocess_func(example):
    inputs = tokenizer(example['prompt'], truncation=True, max_length=CFG.sequence_length, padding='max_length')
    return (
    {
        'input_ids': inputs.input_ids,
        'attention_mask': inputs.attention_mask
    })

train_df['prompt'] = train_df[["original_text", "rewritten_text", "rewrite_prompt"]].progress_apply(gen_prompt, axis=1)
train_ds = Dataset.from_pandas(train_df[['prompt']])
train_ds = train_ds.map(preprocess_func)
train_ds = train_ds.remove_columns(["__index_level_0__", "prompt"])

val_df['prompt'] = val_df[["original_text", "rewritten_text", "rewrite_prompt"]].progress_apply(gen_prompt, axis=1)
val_ds = Dataset.from_pandas(val_df[['prompt']])
val_ds = val_ds.map(preprocess_func)
val_ds = val_ds.remove_columns(["__index_level_0__", "prompt"])



Map: 100%|██████████| 1300/1300 [00:03<00:00, 416.01 examples/s]


In [14]:
import torch
traindata_loader = torch.utils.data.DataLoader(train_ds, batch_size=CFG.train_batch, collate_fn=DataCollatorWithPadding(tokenizer=tokenizer),shuffle=True, num_workers=8)
valdata_loader = torch.utils.data.DataLoader(val_ds, batch_size=CFG.validation_batch, collate_fn=DataCollatorWithPadding(tokenizer=tokenizer), num_workers=8)

# Train!

In [15]:
model.eval()
total_val_loss = 0.0
pbar = tqdm(range(len(valdata_loader)))#, disable=True)
data_iter = iter(valdata_loader)
logging_steps=1
with torch.no_grad():
    for step, batch in enumerate(pbar):
        batch = next(data_iter)
        input_ids, attention_mask = batch.input_ids.to(device), batch.attention_mask.to(device)
        xs.mark_sharding(input_ids, mesh, (0, 1))
        xs.mark_sharding(attention_mask, mesh, (0, 1))
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        total_val_loss = (total_val_loss * step + loss.item()) / (step + 1)
        if (step + 1) % logging_steps == 0:
            pbar.set_postfix({'val_loss': total_val_loss, 'step': step+1})
print(total_val_loss)


100%|██████████| 82/82 [02:56<00:00,  2.15s/it, val_loss=24.3, step=82]

24.263721256721315


In [16]:
!export XLA_USE_BF16=1
def train(
    model, train_data, validation_data=None, train_batch=4, validation_batch=8, epochs=10, logging_steps=1,
    lr=1e-5, 
):
#     def train_model(model, input_ids, attention_mask, optimizer):
#         output = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
#         loss = output.loss
#         loss.backward()
#         optimizer.step()
#         return loss
    
#     compiled_step = torch.compile(model, backend="openxla")
    
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=6000/train_batch*epochs)
    for epoch in range(1, epochs + 1):
        model.train()
        xm.master_print('Epoch {} train begin {}'.format(epoch, test_utils.now()))
        pbar = tqdm(range(len(train_data)))#, disable=True)
        data_iter = iter(train_data)
        total_loss = 0.
#         with torch.autocast("xla", dtype=torch.bfloat16):
        for step, batch in enumerate(pbar):
            batch = next(data_iter)
            optimizer.zero_grad()
            input_ids, attention_mask = batch.input_ids.to(device), batch.attention_mask.to(device)
            xs.mark_sharding(input_ids, mesh, (0, 1)) # Sharding inputs
            xs.mark_sharding(attention_mask, mesh, (0, 1))

#                 loss = compiled_step(model, input_ids, attention_mask, optimizer)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            xm.mark_step()
            total_loss = (total_loss * step + loss.item()) / (step + 1)
            if (step + 1) % logging_steps == 0:
                pbar.set_postfix({'loss': total_loss, 'step': step+1, 'epoch': epoch})
            scheduler.step()
#             if step > 10:
#                 break
        xm.master_print('Epoch {} train end {}, loss={:.3f}'.format(epoch, test_utils.now(), total_loss))
        model.eval()
        total_val_loss = 0.0
        if validation_data is not None:
            pbar = tqdm(range(len(validation_data)))#, disable=True)
            data_iter = iter(validation_data)
            with torch.no_grad():
                for step, batch in enumerate(pbar):
                    batch = next(data_iter)
                    input_ids, attention_mask = batch.input_ids.to(device), batch.attention_mask.to(device)
                    xs.mark_sharding(input_ids, mesh, (0, 1))
                    xs.mark_sharding(attention_mask, mesh, (0, 1))
                    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
                    loss = outputs.loss
                    total_val_loss = (total_val_loss * step + loss.item()) / (step + 1)
                    if (step + 1) % logging_steps == 0:
                        pbar.set_postfix({'val_loss': total_val_loss, 'step': step+1, 'epoch': epoch})

            xm.master_print('Epoch {} test end {}, test val_loss={:.3f}'.format(epoch, test_utils.now(), total_val_loss))
        

        
    model.cpu()
    model.save_pretrained(f"{epoch}-lora_gemmal7b-1.1")


In [ ]:
train(peft_model, traindata_loader, validation_data=valdata_loader, train_batch=CFG.train_batch, epochs=CFG.epochs, lr=8e-5)

Epoch 1 train begin 01:36:08


  0%|          | 0/703 [00:00<?, ?it/s]/usr/local/lib/python3.10/site-packages/torch/autograd/__init__.py:266: UserWarning: aten::reshape: an autograd kernel was not registered to the Autograd key(s) but we are trying to backprop through it. This may lead to silently incorrect behavior. This behavior is deprecated and will be removed in a future version of PyTorch. If your operator is differentiable, please ensure you have registered an autograd kernel to the correct Autograd key (e.g. DispatchKey::Autograd, DispatchKey::CompositeImplicitAutograd). If your operator is not differentiable, or to squash this warning and use the previous behavior, please register torch::CppFunction::makeFallthrough() to DispatchKey::Autograd. (Triggered internally at ../torch/csrc/autograd/autograd_not_implemented_fallback.cpp:63.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
  6%|▌         | 39/703 [06:17<37:21,  3.38s/it, loss=11.6, step=39, epoch=1]  